In [8]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import trim_messages, AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [9]:
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model.invoke(
    [
        HumanMessage(content="Hi, My name is Aryamann Tomar and I am a Sr Data Scientist at GEP Worldwide in Finance Department."),
        AIMessage(content="Hello Aryamann! It's nice to meet you. \n\nAs a Sr Data Scientist, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ])


AIMessage(content="You are Aryamann Tomar, and you are a Senior Data Scientist in the Finance Department at GEP Worldwide.  \n\nIs there anything else you'd like me to remember about you? 😊  I'm here to help!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 110, 'total_tokens': 161, 'completion_time': 0.092727273, 'prompt_time': 0.002808048, 'queue_time': 0.254072521, 'total_time': 0.095535321}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1fbbab5f-b046-4875-9984-2cf2d055228e-0', usage_metadata={'input_tokens': 110, 'output_tokens': 51, 'total_tokens': 161})

In [10]:
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

##### Basic

In [11]:
# with_message_history=RunnableWithMessageHistory(model,get_session_history)
# config={"configurable":{"session_id":"chat1"}}
# response=with_message_history.invoke(
#     [HumanMessage(content="Hi, My name is Aryamann Tomar and I am a Sr Data Scientist at GEP Worldwide in Finance Department.")],
#     config=config
# )
# with_message_history.invoke(
#     [HumanMessage(content="What's my name?")],
#     config=config,
# )

# '''
# #! No Memory of previous conversation
# '''
# config1={"configurable":{"session_id":"chat2"}}
# response=with_message_history.invoke(
#     [HumanMessage(content="Whats my name")],
#     config=config1
# )
# response.content

##### PromptTemplate

In [12]:
# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","You are a helpful assistant. Answer all the question to the nest of your ability"),
#         MessagesPlaceholder(variable_name="messages")
#     ]
# )
# chain=prompt|model
# with_message_history=RunnableWithMessageHistory(chain,get_session_history)
# config = {"configurable": {"session_id": "chat3"}}
# response=with_message_history.invoke(
#     [HumanMessage(content="Hi! My name is Aryamann")],
#     config=config
# )

# response

##### MultipleKeys

In [13]:
prompt = ChatPromptTemplate.from_messages([(
        "system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",),
        MessagesPlaceholder(variable_name="messages")])
chain = prompt|model
# response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Aryamann")],"language":"Hindi"}) 

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, I am Aryamann")],"language":"Hindi"},
    config=config)
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)
response.content

'आपका नाम Aryamann है।  😄  \n'

##### TrimMessages

In [ ]:
trimmer=trim_messages(
    max_tokens=45, strategy="last", token_counter=model, include_system=True,
    allow_partial=False, start_on="human"
)

##### test - trimming

In [28]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob the builder"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)|prompt|model
)
response=chain.invoke({
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    # "messages": messages + [HumanMessage(content="what math problem did i ask")],
    "language":"English"
})
response.content

"As a helpful assistant, I don't have access to your personal preferences, like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream flavor? 😊🍦\n"

##### Now with_message_history

In [31]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}
response = with_message_history.invoke({
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)
response.content

"As an AI, I don't have access to past conversations or personal information about you, so I don't know your name. If you'd like to tell me, I'd be happy to know! 😊  \n"